# JavaScript Reflect Object: Complete Deep Dive

## What is Reflect?

Reflect is a built-in object introduced in ES6 (ECMAScript 2015) that provides methods for interceptable JavaScript operations. It's not a function object, so it's not constructible (you can't use `new Reflect()`). All properties and methods of Reflect are static.

The Reflect object serves as a namespace for static methods that correspond to JavaScript's internal methods. These are the same operations that proxy handlers can intercept.

## Why was Reflect created?

1. **Unified API**: Provides a single namespace for reflection operations that were previously scattered across Object and Function prototypes
2. **Return values**: More consistent return values (often boolean for success/failure)
3. **Proxy complement**: Every Proxy trap has a corresponding Reflect method with identical arguments
4. **Future-proofing**: New reflection methods will be added to Reflect, not Object
5. **Functional programming**: All methods are functions, not syntax (unlike `delete` operator)

## Reflect vs Object vs Proxy

### Reflect vs Object

| Aspect | Reflect | Object |
|--------|---------|--------|
| **Purpose** | Low-level object operations | High-level object utilities and operations |
| **Return values** | Consistent (often boolean) | Mixed (throws errors, returns values) |
| **Error handling** | Returns false on failure | Often throws TypeError |
| **Method behavior** | Mirrors internal operations | Higher-level abstractions |
| **Future methods** | New reflection methods go here | Legacy, won't get new reflection methods |

#### Key Differences in Behavior

```javascript
// Object.defineProperty throws on failure
try {
  Object.defineProperty(Object.freeze({}), 'prop', {value: 1});
} catch(e) {
  console.log('Object threw error');
}

// Reflect.defineProperty returns false
const result = Reflect.defineProperty(Object.freeze({}), 'prop', {value: 1});
console.log(result); // false

// Object.getOwnPropertyDescriptor only works on objects
Object.getOwnPropertyDescriptor('string', 'length'); // throws TypeError in strict mode

// Reflect.getOwnPropertyDescriptor coerces to object
Reflect.getOwnPropertyDescriptor('string', 'length'); // works fine
```

### Reflect vs Proxy

| Aspect | Reflect | Proxy |
|--------|---------|-------|
| **Role** | Performs operations | Intercepts operations |
| **Usage** | Direct method calls | Wraps objects with traps |
| **Relationship** | Default behavior for Proxy traps | Defines custom behavior |
| **Methods/Traps** | 13 static methods | 13 handler traps |

The methods of Reflect correspond 1:1 with Proxy handler traps:

```javascript
const proxy = new Proxy(target, {
  get(target, property, receiver) {
    // Reflect.get provides the default behavior
    return Reflect.get(target, property, receiver);
  }
});
```

## All Reflect Methods (Complete Reference)

### 1. Reflect.apply()

**Signature:** `Reflect.apply(target, thisArgument, argumentsList)`

**Purpose:** Calls a target function with a given this value and arguments.

```javascript
function greet(prefix, suffix) {
  return `${prefix} ${this.name} ${suffix}`;
}

const person = { name: 'Alice' };
const result = Reflect.apply(greet, person, ['Hello', '!']);
console.log(result); // "Hello Alice !"

// Equivalent to:
// greet.apply(person, ['Hello', '!'])
```

### 2. Reflect.construct()

**Signature:** `Reflect.construct(target, argumentsList[, newTarget])`

**Purpose:** Acts like the `new` operator, but as a function. Optional newTarget changes prototype.

```javascript
class Person {
  constructor(name, age) {
    this.name = name;
    this.age = age;
  }
}

class Employee extends Person {
  constructor(name, age, id) {
    super(name, age);
    this.id = id;
  }
}

// Normal construction
const p1 = Reflect.construct(Person, ['Alice', 30]);
console.log(p1 instanceof Person); // true

// With different prototype
const p2 = Reflect.construct(Person, ['Bob', 25], Employee);
console.log(p2 instanceof Employee); // true
console.log(p2 instanceof Person); // true
```

### 3. Reflect.defineProperty()

**Signature:** `Reflect.defineProperty(target, propertyKey, attributes)`

**Purpose:** Defines a new property or modifies an existing one. Returns boolean.

```javascript
const obj = {};

const success = Reflect.defineProperty(obj, 'property', {
  value: 42,
  writable: true,
  enumerable: true,
  configurable: true
});

console.log(success); // true
console.log(obj.property); // 42

// Failing case
const frozen = Object.freeze({});
const failed = Reflect.defineProperty(frozen, 'prop', {value: 1});
console.log(failed); // false
```

### 4. Reflect.deleteProperty()

**Signature:** `Reflect.deleteProperty(target, propertyKey)`

**Purpose:** Deletes a property. Equivalent to `delete` operator but as a function.

```javascript
const obj = { foo: 'bar', baz: 42 };

Reflect.deleteProperty(obj, 'foo');
console.log(obj); // { baz: 42 }

// With non-configurable property
const obj2 = {};
Object.defineProperty(obj2, 'frozen', {
  value: 'immutable',
  configurable: false
});

const deleted = Reflect.deleteProperty(obj2, 'frozen');
console.log(deleted); // false
console.log(obj2.frozen); // 'immutable'
```

### 5. Reflect.get()

**Signature:** `Reflect.get(target, propertyKey[, receiver])`

**Purpose:** Gets a property value. The receiver is the `this` value for getters.

```javascript
const obj = {
  x: 1,
  y: 2,
  get sum() {
    return this.x + this.y;
  }
};

console.log(Reflect.get(obj, 'x')); // 1
console.log(Reflect.get(obj, 'sum')); // 3

// With different receiver
const receiver = { x: 10, y: 20 };
console.log(Reflect.get(obj, 'sum', receiver)); // 30

// Array length
console.log(Reflect.get([1, 2, 3], 'length')); // 3
```

### 6. Reflect.getOwnPropertyDescriptor()

**Signature:** `Reflect.getOwnPropertyDescriptor(target, propertyKey)`

**Purpose:** Returns a property descriptor for an own property.

```javascript
const obj = { foo: 'bar' };

const descriptor = Reflect.getOwnPropertyDescriptor(obj, 'foo');
console.log(descriptor);
// { value: 'bar', writable: true, enumerable: true, configurable: true }

// With getter/setter
const obj2 = {
  get prop() { return 'getter'; },
  set prop(val) { console.log('setter:', val); }
};

const desc2 = Reflect.getOwnPropertyDescriptor(obj2, 'prop');
console.log(desc2);
// { get: [Function: get prop], set: [Function: set prop], enumerable: true, configurable: true }
```

### 7. Reflect.getPrototypeOf()

**Signature:** `Reflect.getPrototypeOf(target)`

**Purpose:** Returns the prototype of an object.

```javascript
class Animal {}
class Dog extends Animal {}

const myDog = new Dog();

console.log(Reflect.getPrototypeOf(myDog) === Dog.prototype); // true
console.log(Reflect.getPrototypeOf(Dog.prototype) === Animal.prototype); // true

// With plain object
const obj = { a: 1 };
console.log(Reflect.getPrototypeOf(obj) === Object.prototype); // true
```

### 8. Reflect.has()

**Signature:** `Reflect.has(target, propertyKey)`

**Purpose:** Checks if an object has a property. Works like the `in` operator.

```javascript
const obj = { foo: 'bar' };
const arr = [1, 2, 3];

console.log(Reflect.has(obj, 'foo')); // true
console.log(Reflect.has(obj, 'bar')); // false

// Checks prototype chain
console.log(Reflect.has(obj, 'toString')); // true

// With arrays
console.log(Reflect.has(arr, 0)); // true
console.log(Reflect.has(arr, 'length')); // true
```

### 9. Reflect.isExtensible()

**Signature:** `Reflect.isExtensible(target)`

**Purpose:** Determines if an object is extensible (can have new properties added).

```javascript
const obj = {};
console.log(Reflect.isExtensible(obj)); // true

Object.preventExtensions(obj);
console.log(Reflect.isExtensible(obj)); // false

const sealed = Object.seal({});
console.log(Reflect.isExtensible(sealed)); // false

const frozen = Object.freeze({});
console.log(Reflect.isExtensible(frozen)); // false
```

### 10. Reflect.ownKeys()

**Signature:** `Reflect.ownKeys(target)`

**Purpose:** Returns an array of all own property keys (strings and symbols).

```javascript
const sym = Symbol('sym');
const obj = {
  foo: 1,
  [sym]: 2,
  bar: 3
};

// Define non-enumerable property
Object.defineProperty(obj, 'hidden', {
  value: 'secret',
  enumerable: false
});

console.log(Reflect.ownKeys(obj));
// ['foo', 'bar', 'hidden', Symbol(sym)]

// Compare with other methods:
console.log(Object.keys(obj)); // ['foo', 'bar'] - only enumerable strings
console.log(Object.getOwnPropertyNames(obj)); // ['foo', 'bar', 'hidden'] - all strings
console.log(Object.getOwnPropertySymbols(obj)); // [Symbol(sym)] - only symbols
```

### 11. Reflect.preventExtensions()

**Signature:** `Reflect.preventExtensions(target)`

**Purpose:** Prevents new properties from being added to an object.

```javascript
const obj = { existing: 'property' };

const result = Reflect.preventExtensions(obj);
console.log(result); // true

// Cannot add new properties
obj.newProp = 'value';
console.log(obj.newProp); // undefined (in strict mode, throws error)

// Can still modify existing
obj.existing = 'modified';
console.log(obj.existing); // 'modified'

// Returns false for primitives (Object.preventExtensions throws)
console.log(Reflect.preventExtensions(1)); // false
```

### 12. Reflect.set()

**Signature:** `Reflect.set(target, propertyKey, value[, receiver])`

**Purpose:** Sets a property value. Returns boolean indicating success.

```javascript
const obj = {};

Reflect.set(obj, 'prop', 'value');
console.log(obj.prop); // 'value'

// With setter
const obj2 = {
  _value: 0,
  set prop(val) {
    console.log(`Setting to ${val}`);
    this._value = val;
  }
};

Reflect.set(obj2, 'prop', 42); // logs: "Setting to 42"

// With different receiver
const receiver = { _value: 'receiver' };
Reflect.set(obj2, 'prop', 100, receiver); // Sets receiver._value, not obj2._value
console.log(receiver._value); // 100
console.log(obj2._value); // 42
```

### 13. Reflect.setPrototypeOf()

**Signature:** `Reflect.setPrototypeOf(target, prototype)`

**Purpose:** Sets the prototype of an object. Returns boolean.

```javascript
const obj = {};
const proto = { foo: 'bar' };

const result = Reflect.setPrototypeOf(obj, proto);
console.log(result); // true
console.log(obj.foo); // 'bar'

// Failing case - non-extensible object
const frozen = Object.freeze({});
const failed = Reflect.setPrototypeOf(frozen, {});
console.log(failed); // false
```

## Common Use Cases and Patterns

### 1. Default Proxy Handler

```javascript
const handler = {
  get(target, property, receiver) {
    console.log(`Getting ${property}`);
    return Reflect.get(target, property, receiver);
  },
  set(target, property, value, receiver) {
    console.log(`Setting ${property} to ${value}`);
    return Reflect.set(target, property, value, receiver);
  }
};

const proxy = new Proxy({}, handler);
```

### 2. Property Validation

```javascript
function createValidatedObject(validators) {
  return new Proxy({}, {
    set(target, property, value, receiver) {
      if (property in validators) {
        if (!validators[property](value)) {
          throw new TypeError(`Invalid value for ${property}`);
        }
      }
      return Reflect.set(target, property, value, receiver);
    }
  });
}

const person = createValidatedObject({
  age: value => typeof value === 'number' && value > 0 && value < 150,
  name: value => typeof value === 'string' && value.length > 0
});
```

### 3. Method Binding

```javascript
function autoBind(obj) {
  const cache = new WeakMap();
  
  return new Proxy(obj, {
    get(target, property, receiver) {
      const value = Reflect.get(target, property, receiver);
      
      if (typeof value === 'function') {
        if (!cache.has(value)) {
          cache.set(value, value.bind(receiver));
        }
        return cache.get(value);
      }
      
      return value;
    }
  });
}
```

### 4. Observable Objects

```javascript
function observable(target, onChange) {
  return new Proxy(target, {
    set(obj, prop, value, receiver) {
      const oldValue = obj[prop];
      const result = Reflect.set(obj, prop, value, receiver);
      
      if (result && oldValue !== value) {
        onChange(prop, oldValue, value);
      }
      
      return result;
    },
    deleteProperty(obj, prop) {
      const oldValue = obj[prop];
      const result = Reflect.deleteProperty(obj, prop);
      
      if (result) {
        onChange(prop, oldValue, undefined);
      }
      
      return result;
    }
  });
}
```

## Best Practices

1. **Use Reflect in Proxy handlers**: Always use corresponding Reflect methods for default behavior
2. **Check return values**: Reflect methods return booleans for success/failure
3. **Prefer Reflect over Object**: For low-level operations, Reflect provides more consistent behavior
4. **Handle receiver properly**: In get/set traps, always pass the receiver parameter
5. **Type checking**: Reflect methods throw TypeError for non-objects (except getOwnPropertyDescriptor)

## Browser Support

Reflect is fully supported in:
- Chrome 49+
- Firefox 42+
- Safari 10+
- Edge 12+
- Node.js 6+

## Summary

Reflect provides a unified, consistent API for meta-programming operations in JavaScript. It's the perfect companion to Proxy, offering default implementations for all proxy traps. While Object methods still work, Reflect offers better error handling, more consistent return values, and is the future-forward approach for reflection operations in JavaScript.